In [1]:
import numpy as np
import os
import sys
import math

# ----- class for snapshot header ----- 

class snapshot_header:
    def __init__(self, filename):
        if (not os.path.exists(filename)):
            print("file not found:", filename)
            sys.exit()
      
        self.filename = filename  
        f = open(filename,'rb')    
        blocksize = np.fromfile(f,dtype=np.int32,count=1)
        if blocksize[0] == 8:
            swap = 0
            format = 2
        elif blocksize[0] == 256:
            swap = 0
            format = 1  
        else:
            blocksize.byteswap(True)
            if blocksize[0] == 8:
                swap = 1
                format = 2
            elif blocksize[0] == 256:
                swap = 1
                format = 1
            else:
                print("incorrect file format encountered when reading header of", filename)
                sys.exit()
    
        self.format = format
        self.swap = swap
    
        if format==2:
            f.seek(16, os.SEEK_CUR)
    
        self.npart = np.fromfile(f,dtype=np.int32,count=6)
        self.massarr = np.fromfile(f,dtype=np.float64,count=6)
        self.time = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.redshift = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.sfr = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.feedback = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.nall = np.fromfile(f,dtype=np.int32,count=6)
        self.cooling = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.filenum = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.boxsize = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.omega_m = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.omega_l = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.hubble = (np.fromfile(f,dtype=np.float64,count=1))[0]
    
        if swap:
            self.npart.byteswap(True)
            self.massarr.byteswap(True)
            self.time = self.time.byteswap()
            self.redshift = self.redshift.byteswap()
            self.sfr = self.sfr.byteswap()
            self.feedback = self.feedback.byteswap()
            self.nall.byteswap(True)
            self.cooling = self.cooling.byteswap()
            self.filenum = self.filenum.byteswap()
            self.boxsize = self.boxsize.byteswap()
            self.omega_m = self.omega_m.byteswap()
            self.omega_l = self.omega_l.byteswap()
            self.hubble = self.hubble.byteswap()
     
        f.close()
 


#----- find offset and size of data block ----- 

def find_block(filename, format, swap, block, block_num, only_list_blocks=False):
    if (not os.path.exists(filename)):
        print("file not found:", filename)
        sys.exit()
            
    f = open(filename,'rb')
    f.seek(0, os.SEEK_END)
    filesize = f.tell()
    f.seek(0, os.SEEK_SET)
  
    found = False
    curblock_num = 1
    while ((not found) and (f.tell()<filesize)):
        if format==2:
            f.seek(4, os.SEEK_CUR)
            curblock = f.read(4)
            if (block == curblock):
                found = True
            f.seek(8, os.SEEK_CUR)  
        else:
            if curblock_num==block_num:
                found = True
        
        curblocksize = (np.fromfile(f,dtype=np.int32,count=1))[0]
        if swap:
            curblocksize = curblocksize.byteswap()
    
    # - print some debug info about found data blocks -
    #if format==2:
    #  print curblock, curblock_num, curblocksize
    #else:
    #  print curblock_num, curblocksize
    
        if only_list_blocks:
            print(curblock_num,curblock,f.tell(),curblocksize)
            found = False
    
        if found:
            blocksize = curblocksize
            offset = f.tell()
        else:
            f.seek(curblocksize, os.SEEK_CUR)
            blocksize_check = (np.fromfile(f,dtype=np.int32,count=1))[0]
            if swap: blocksize_check = blocksize_check.byteswap()
            if (curblocksize != blocksize_check):
                print("something wrong")
                sys.exit()
            curblock_num += 1
      
    f.close()
      
    if ((not found) and (not only_list_blocks)):
        print("Error: block not found")
        sys.exit()
    
    if (not only_list_blocks):
        return offset,blocksize
 
# ----- read data block -----
 
def read_block(filename, block, parttype=-1, physical_velocities=True, arepo=0, no_masses=False, verbose=False):
    if (verbose):
	    print("reading block", block)
  
    blockadd=0
    blocksub=0
  
    if arepo==0:
        if (verbose):	
	        print("Gadget format")
        blockadd=0
    if arepo==1:
        if (verbose):	
	        print("Arepo format")
        blockadd=1	
    if arepo==2:
        if (verbose):
	        print("Arepo extended format")
        blockadd=4	
    if no_masses==True:
        if (verbose):	
	        print("No mass block present")    
        blocksub=1
		 
    if parttype not in [-1,0,1,2,3,4,5]:
        print("wrong parttype given")
        sys.exit()
  
    if os.path.exists(filename):
        curfilename = filename
    elif os.path.exists(filename+".0"):
        curfilename = filename+".0"
    else:
        print("file not found:", filename)
        print("and:", curfilename)
        sys.exit()
  
    head = snapshot_header(curfilename)
    format = head.format
    swap = head.swap
    npart = head.npart
    massarr = head.massarr
    nall = head.nall
    filenum = head.filenum
    redshift = head.redshift
    time = head.time
    del head
  
  # - description of data blocks -
  # add or change blocks as needed for your Gadget version
    data_for_type = np.zeros(6,bool) # should be set to "True" below for the species for which data is stored in the data block
    dt = np.float32 # data type of the data in the block
    if block=="POS ":
        data_for_type[:] = True
        dt = np.dtype((np.float32,3))
        block_num = 2
    elif block=="VEL ":
        data_for_type[:] = True
        dt = np.dtype((np.float32,3))
        block_num = 3
    elif block=="ID  ":
        data_for_type[:] = True
        dt = np.uint32
        block_num = 4
    elif block=="MASS":
        data_for_type[np.where(massarr==0)] = True
        block_num = 5
        if parttype>=0 and massarr[parttype]>0:   
            if (verbose):	
	            print("filling masses according to massarr")   
            return np.ones(nall[parttype],dtype=dt)*massarr[parttype]
    elif block=="U   ":
        data_for_type[:] = True
        dt = np.dtype((np.float32))
        block_num = 6#-blocksub
    elif block=="RHO ":
        data_for_type[0] = True
        block_num = 7-blocksub
    elif block=="VOL ":
        data_for_type[0] = True
        block_num = 8-blocksub 
    elif block=="CMCE":
        data_for_type[0] = True
        dt = np.dtype((np.float32,3))
        block_num = 9-blocksub 
    elif block=="AREA":
        data_for_type[0] = True
        block_num = 10-blocksub
    elif block=="NFAC":
        data_for_type[0] = True
        dt = np.dtype(np.int32)	
        block_num = 11-blocksub
    elif block=="NE  ":
        data_for_type[0] = True
        block_num = 8+blockadd-blocksub
    elif block=="NH  ":
        data_for_type[0] = True
        block_num = 9+blockadd-blocksub
    elif block=="HSML":
        data_for_type[0] = True
        block_num = 10+blockadd-blocksub
    elif block=="SFR ":
        data_for_type[0] = True
        block_num = 11+blockadd-blocksub
    elif block=="AGE ":
        data_for_type[4] = True
        block_num = 12+blockadd-blocksub
    elif block=="Z   ":
        data_for_type[0] = True
        data_for_type[4] = True
        block_num = 13+blockadd-blocksub
    elif block=="BHMA":
        data_for_type[5] = True
        block_num = 14+blockadd-blocksub
    elif block=="BHMD":
        data_for_type[5] = True
        block_num = 15+blockadd-blocksub
    elif block=="COOR":
        data_for_type[0] = True
        block_num = -1 
    else:
        print("Sorry! Block type", block, "not known!")
        sys.exit()
  # - end of block description -

    if (block_num < 0 and format==1):
        print("Sorry! Block number of", block, "not known! Unable to read this block from format 1 file!")
        sys.exit() 
    
    actual_data_for_type = np.copy(data_for_type)  
    if parttype >= 0:
        actual_data_for_type[:] = False
        actual_data_for_type[parttype] = True
        if data_for_type[parttype]==False:
            print("Error: no data for specified particle type", parttype, "in the block", block)   
            sys.exit()
    elif block=="MASS":
        actual_data_for_type[:] = True  
    
    allpartnum = np.int64(0)
    species_offset = np.zeros(6,np.int64)
    for j in range(6):
        species_offset[j] = allpartnum
        if actual_data_for_type[j]:
            allpartnum += nall[j]
    filenum=1  
    for i in range(filenum): # main loop over files
        if filenum>1:
            curfilename = filename+"."+str(i)
      
        if i>0:
            head = snapshot_header(curfilename)
            npart = head.npart  
            del head
      
        curpartnum = np.int32(0)
        cur_species_offset = np.zeros(6,np.int64)
        for j in range(6):
            cur_species_offset[j] = curpartnum
            if data_for_type[j]:
                curpartnum += npart[j]
    
        if parttype>=0:
            actual_curpartnum = npart[parttype]      
            add_offset = cur_species_offset[parttype] 
        else:
            actual_curpartnum = curpartnum
            add_offset = np.int32(0)
      
        offset,blocksize = find_block(curfilename,format,swap,block,block_num)
    
        if i==0: # fix data type for ID if long IDs are used
            if block=="ID  ":
                if blocksize == np.dtype(dt).itemsize*curpartnum * 2:
                    dt = np.uint64 
        
        if np.dtype(dt).itemsize*curpartnum != blocksize:
            print("something wrong with blocksize! expected =",np.dtype(dt).itemsize*curpartnum,"actual =",blocksize)
            sys.exit()
    
        f = open(curfilename,'rb')
        f.seek(offset + add_offset*np.dtype(dt).itemsize, os.SEEK_CUR)  
        curdat = np.fromfile(f,dtype=dt,count=actual_curpartnum) # read data
        f.close()  
        if swap:
            curdat.byteswap(True)  
      
        if i==0:
            data = np.empty(allpartnum,dt)
    
        for j in range(6):
            if actual_data_for_type[j]:
                if block=="MASS" and massarr[j]>0: # add mass block for particles for which the mass is specified in the snapshot header
                    data[species_offset[j]:species_offset[j]+npart[j]] = massarr[j]
                else:
                    if parttype>=0:
                        data[species_offset[j]:species_offset[j]+npart[j]] = curdat
                    else:
                        data[species_offset[j]:species_offset[j]+npart[j]] = curdat[cur_species_offset[j]:cur_species_offset[j]+npart[j]]
                species_offset[j] += npart[j]

        del curdat

    if physical_velocities and block=="VEL " and redshift!=0:
        data *= math.sqrt(time)

    return data
  
# ----- list all data blocks in a format 2 snapshot file -----

def list_format2_blocks(filename):
    if (not os.path.exists(filename)):
        print("file not found:", filename)
        sys.exit()
  
    head = snapshot_header(filename)
    format = head.format
    swap = head.swap
    del head
  
    if (format != 2):
        print("not a format 2 snapshot file")
        sys.exit()
            
    print("#   BLOCK   OFFSET   SIZE")
    print("-------------------------")
  
    find_block(filename, format, swap, "XXXX", 0, only_list_blocks=True)
  
    print("-------------------------")


In [12]:
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats

%matplotlib inline

header =snapshot_header("/home/douglas/Chervin/snap_811")

mass1 =read_block("/home/douglas/Chervin/snap_811","MASS",parttype=1)
pos1 =read_block("/home/douglas/Chervin/snap_811","POS ",parttype=1)
vel1 =read_block("/home/douglas/Chervin/snap_811","VEL ",parttype=1)

mass2 =read_block("/home/douglas/Chervin/snap_811","MASS",parttype=2)
pos2 =read_block("/home/douglas/Chervin/snap_811","POS ",parttype=2)
vel2 =read_block("/home/douglas/Chervin/snap_811","VEL ",parttype=2)

mass3 =read_block("/home/douglas/Chervin/snap_811","MASS",parttype=3)
pos3 =read_block("/home/douglas/Chervin/snap_811","POS ",parttype=3)
vel3 =read_block("/home/douglas/Chervin/snap_811","VEL ",parttype=3)

pos1 = pos1/1000
pos3 = pos3/1000

In [27]:
mass1 = mass1* 1000000000
mass2 = mass2* 1000000000
mass3 = mass3* 1000000000

In [28]:
import agama

agama.setUnits(mass=1, length=1, velocity=1)

dark = agama.Potential(type = "Multipole",particles=(pos1, mass1),symmetry='a', gridsizeR=20, lmax=2)

disk = agama.Potential(type = "CylSpline",particles=(pos2, mass2),gridsizeR=20, gridsizeZ=20,
                        mmax=0, Rmin=0.2, symmetry='a',Rmax=70, Zmin=0.02, Zmax=10)
                       
bulge = agama.Potential(type = "Multipole",particles=(pos3, mass3),symmetry='a',gridsizeR=20,lmax=2)

In [29]:
potential = agama.Potential(dark, disk, bulge)
af = agama.ActionFinder(potential, interp=False)

In [30]:
bulgepoints = np.hstack((pos3,vel3))
diskpoints = np.hstack((pos2,vel2))
dmpoints = np.hstack((pos1,vel1))


Bulg_actions, Bulg_angles, Bulg_freq = af(bulgepoints, angles=True)
Disk_actions, Disk_angles, Disk_freq = af(diskpoints, angles=True)
DM_actions, DM_angles, DM_freq = af(dmpoints, angles=True)

In [31]:
Jrdisk = Disk_actions[:, 0]
Jzdisk = Disk_actions[:, 1]
Jphidisk = Disk_actions[:, 2]

Jrbulg = Bulg_actions[:, 0]
Jzbulg = Bulg_actions[:, 1]
Jphibulg = Bulg_actions[:, 2]

Jrdm = DM_actions[:, 0]
Jzdm = DM_actions[:, 1]
Jphidm = DM_actions[:, 2]

Trdisk = Disk_angles[:, 0]
Tzdisk = Disk_angles[:, 1]
Tphidisk = Disk_angles[:, 2]

Trbulg = Bulg_angles[:, 0]
Tzbulg = Bulg_angles[:, 1]
Tphibulg = Bulg_angles[:, 2]

Trdm = DM_angles[:, 0]
Tzdm = DM_angles[:, 1]
Tphidm = DM_angles[:, 2]

Ordisk = Disk_freq[:, 0]
Ozdisk = Disk_freq[:, 1]
Ophidisk = Disk_freq[:, 2]

Orbulg = Bulg_freq[:, 0]
Ozbulg = Bulg_freq[:, 1]
Ophibulg = Bulg_freq[:, 2]

Ordm = DM_freq[:, 0]
Ozdm = DM_freq[:, 1]
Ophidm = DM_freq[:, 2]

In [37]:
header =snapshot_header("/home/douglas/Chervin/snap_019")

dm_mass_019 =read_block("/home/douglas/Chervin/snap_019","MASS",parttype=1)
dm_pos_019 =read_block("/home/douglas/Chervin/snap_019","POS ",parttype=1)
dm_vel_019 =read_block("/home/douglas/Chervin/snap_019","VEL ",parttype=1)

disk_mass_019 =read_block("/home/douglas/Chervin/snap_019","MASS",parttype=2)
disk_pos_019 =read_block("/home/douglas/Chervin/snap_019","POS ",parttype=2)
disk_vel_019 =read_block("/home/douglas/Chervin/snap_019","VEL ",parttype=2)

bulg_mass_019 =read_block("/home/douglas/Chervin/snap_019","MASS",parttype=3)
bulg_pos_019 =read_block("/home/douglas/Chervin/snap_019","POS ",parttype=3)
bulg_vel_019 =read_block("/home/douglas/Chervin/snap_019","VEL ",parttype=3)

dm_pos_019 = dm_pos_019/1000
bulg_pos_019 = bulg_pos_019/1000
dm_mass_019 = dm_mass_019 * 1000000000
disk_mass_019 = disk_mass_019 * 1000000000
bulg_mass_019 = bulg_mass_019 * 1000000000

In [38]:
agama.setUnits(mass=1, length=1, velocity=1)

dark_019 = agama.Potential(type = "Multipole",particles=(dm_pos_019, dm_mass_019),symmetry='a', gridsizeR=20, lmax=2)

disk_019 = agama.Potential(type = "CylSpline",particles=(disk_pos_019, disk_mass_019),gridsizeR=20, gridsizeZ=20,
                        mmax=0, Rmin=0.2, symmetry='a',Rmax=70, Zmin=0.02, Zmax=10)
                       
bulge_019 = agama.Potential(type = "Multipole",particles=(bulg_pos_019, bulg_mass_019),symmetry='a',gridsizeR=20,lmax=2)

In [39]:
potential_019 = agama.Potential(dark_019, disk_019, bulge_019)
af_019 = agama.ActionFinder(potential, interp=False)

bulgepoints_019 = np.hstack((bulg_pos_019,bulg_vel_019))
diskpoints_019 = np.hstack((disk_pos_019,disk_vel_019))
dmpoints_019 = np.hstack((dm_pos_019,dm_vel_019))


Bulg_actions_019, Bulg_angles_019, Bulg_freq_019 = af(bulgepoints_019, angles=True)
Disk_actions_019, Disk_angles_019, Disk_freq_019 = af(diskpoints_019, angles=True)
DM_actions_019, DM_angles_019, DM_freq_019 = af(dmpoints_019, angles=True)

In [40]:
Jrdisk_019 = Disk_actions_019[:, 0]
Jzdisk_019 = Disk_actions_019[:, 1]
Jphidisk_019 = Disk_actions_019[:, 2]

Jrbulg_019 = Bulg_actions_019[:, 0]
Jzbulg_019 = Bulg_actions_019[:, 1]
Jphibulg_019 = Bulg_actions_019[:, 2]

Jrdm_019 = DM_actions_019[:, 0]
Jzdm_019 = DM_actions_019[:, 1]
Jphidm_019 = DM_actions_019[:, 2]

Trdisk_019 = Disk_angles_019[:, 0]
Tzdisk_019 = Disk_angles_019[:, 1]
Tphidisk_019 = Disk_angles_019[:, 2]

Trbulg_019 = Bulg_angles_019[:, 0]
Tzbulg_019 = Bulg_angles_019[:, 1]
Tphibulg_019 = Bulg_angles_019[:, 2]

Trdm_019 = DM_angles_019[:, 0]
Tzdm_019 = DM_angles_019[:, 1]
Tphidm_019 = DM_angles_019[:, 2]

Ordisk_019 = Disk_freq_019[:, 0]
Ozdisk_019 = Disk_freq_019[:, 1]
Ophidisk_019 = Disk_freq_019[:, 2]

Orbulg_019 = Bulg_freq_019[:, 0]
Ozbulg_019 = Bulg_freq_019[:, 1]
Ophibulg_019 = Bulg_freq_019[:, 2]

Ordm_019 = DM_freq_019[:, 0]
Ozdm_019 = DM_freq_019[:, 1]
Ophidm_019 = DM_freq_019[:, 2]

In [41]:
from astropy.io import fits
c1 = fits.Column(name='Jr', array=np.array(Jrdisk), format='D', unit = 'kpc2 Myr-1')
c2 = fits.Column(name='Jphi', array=np.array(Jphidisk), format='D', unit = 'kpc2 Myr-1')
c3 = fits.Column(name='Jz', array=np.array(Jzdisk), format='D', unit ='kpc2 Myr-1')
c4 = fits.Column(name='Theta_r', array=np.array(Trdisk), format='D', unit = 'rad')
c5 = fits.Column(name='Theta_phi', array=np.array(Tphidisk), format='D', unit = 'rad')
c6 = fits.Column(name='Theta_z', array=np.array(Tzdisk), format='D', unit='rad')
c7 = fits.Column(name='Freq_r', array=np.array(Ordisk), format='D', unit='Myr-1')
c8 = fits.Column(name='Freq_phi', array=np.array(Ophidisk), format='D',unit='Myr-1')
c9 = fits.Column(name='Freq_z', array=np.array(Ozdisk), format='D', unit='Myr-1')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9])

t.writeto('DiskActions_811.fits')

In [42]:
c1 = fits.Column(name='Jr', array=np.array(Jrdm), format='D', unit = 'kpc2 Myr-1')
c2 = fits.Column(name='Jphi', array=np.array(Jphidm), format='D', unit = 'kpc2 Myr-1')
c3 = fits.Column(name='Jz', array=np.array(Jzdm), format='D', unit ='kpc2 Myr-1')
c4 = fits.Column(name='Theta_r', array=np.array(Trdm), format='D', unit = 'rad')
c5 = fits.Column(name='Theta_phi', array=np.array(Tphidm), format='D', unit = 'rad')
c6 = fits.Column(name='Theta_z', array=np.array(Tzdm), format='D', unit='rad')
c7 = fits.Column(name='Freq_r', array=np.array(Ordm), format='D', unit='Myr-1')
c8 = fits.Column(name='Freq_phi', array=np.array(Ophidm), format='D',unit='Myr-1')
c9 = fits.Column(name='Freq_z', array=np.array(Ozdm), format='D', unit='Myr-1')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9])

t.writeto('DMActions_811.fits')

In [43]:
c1 = fits.Column(name='Jr', array=np.array(Jrbulg), format='D', unit = 'kpc2 Myr-1')
c2 = fits.Column(name='Jphi', array=np.array(Jphibulg), format='D', unit = 'kpc2 Myr-1')
c3 = fits.Column(name='Jz', array=np.array(Jzbulg), format='D', unit ='kpc2 Myr-1')
c4 = fits.Column(name='Theta_r', array=np.array(Trbulg), format='D', unit = 'rad')
c5 = fits.Column(name='Theta_phi', array=np.array(Tphibulg), format='D', unit = 'rad')
c6 = fits.Column(name='Theta_z', array=np.array(Tzbulg), format='D', unit='rad')
c7 = fits.Column(name='Freq_r', array=np.array(Orbulg), format='D', unit='Myr-1')
c8 = fits.Column(name='Freq_phi', array=np.array(Ophibulg), format='D',unit='Myr-1')
c9 = fits.Column(name='Freq_z', array=np.array(Ozbulg), format='D', unit='Myr-1')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9])

t.writeto('BulgeActions_811.fits')

In [44]:
c1 = fits.Column(name='Jr', array=np.array(Jrdisk_019), format='D', unit = 'kpc2 Myr-1')
c2 = fits.Column(name='Jphi', array=np.array(Jphidisk_019), format='D', unit = 'kpc2 Myr-1')
c3 = fits.Column(name='Jz', array=np.array(Jzdisk_019), format='D', unit ='kpc2 Myr-1')
c4 = fits.Column(name='Theta_r', array=np.array(Trdisk_019), format='D', unit = 'rad')
c5 = fits.Column(name='Theta_phi', array=np.array(Tphidisk_019), format='D', unit = 'rad')
c6 = fits.Column(name='Theta_z', array=np.array(Tzdisk_019), format='D', unit='rad')
c7 = fits.Column(name='Freq_r', array=np.array(Ordisk_019), format='D', unit='Myr-1')
c8 = fits.Column(name='Freq_phi', array=np.array(Ophidisk_019), format='D',unit='Myr-1')
c9 = fits.Column(name='Freq_z', array=np.array(Ozdisk_019), format='D', unit='Myr-1')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9])

t.writeto('DiskActions_019.fits')

In [45]:
c1 = fits.Column(name='Jr', array=np.array(Jrdm_019), format='D', unit = 'kpc2 Myr-1')
c2 = fits.Column(name='Jphi', array=np.array(Jphidm_019), format='D', unit = 'kpc2 Myr-1')
c3 = fits.Column(name='Jz', array=np.array(Jzdm_019), format='D', unit ='kpc2 Myr-1')
c4 = fits.Column(name='Theta_r', array=np.array(Trdm_019), format='D', unit = 'rad')
c5 = fits.Column(name='Theta_phi', array=np.array(Tphidm_019), format='D', unit = 'rad')
c6 = fits.Column(name='Theta_z', array=np.array(Tzdm_019), format='D', unit='rad')
c7 = fits.Column(name='Freq_r', array=np.array(Ordm_019), format='D', unit='Myr-1')
c8 = fits.Column(name='Freq_phi', array=np.array(Ophidm_019), format='D',unit='Myr-1')
c9 = fits.Column(name='Freq_z', array=np.array(Ozdm_019), format='D', unit='Myr-1')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9])

t.writeto('DMActions_019.fits')

In [46]:
c1 = fits.Column(name='Jr', array=np.array(Jrbulg_019), format='D', unit = 'kpc2 Myr-1')
c2 = fits.Column(name='Jphi', array=np.array(Jphibulg_019), format='D', unit = 'kpc2 Myr-1')
c3 = fits.Column(name='Jz', array=np.array(Jzbulg_019), format='D', unit ='kpc2 Myr-1')
c4 = fits.Column(name='Theta_r', array=np.array(Trbulg_019), format='D', unit = 'rad')
c5 = fits.Column(name='Theta_phi', array=np.array(Tphibulg_019), format='D', unit = 'rad')
c6 = fits.Column(name='Theta_z', array=np.array(Tzbulg_019), format='D', unit='rad')
c7 = fits.Column(name='Freq_r', array=np.array(Orbulg_019), format='D', unit='Myr-1')
c8 = fits.Column(name='Freq_phi', array=np.array(Ophibulg_019), format='D',unit='Myr-1')
c9 = fits.Column(name='Freq_z', array=np.array(Ozbulg_019), format='D', unit='Myr-1')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9])

t.writeto('BulgeActions_019.fits')

In [47]:
from astropy.io import fits
c1 = fits.Column(name='X', array=np.array(pos3[:,0]), format='D', unit = 'kpc')
c2 = fits.Column(name='Y', array=np.array(pos3[:,1]), format='D', unit = 'kpc')
c3 = fits.Column(name='Z', array=np.array(pos3[:,2]), format='D', unit ='kpc')
c4 = fits.Column(name='U', array=np.array(vel3[:,0]), format='D', unit = 'km/s')
c5 = fits.Column(name='V', array=np.array(vel3[:,1]), format='D', unit = 'km/s')
c6 = fits.Column(name='W', array=np.array(vel3[:,2]), format='D', unit='km/s')
c7 = fits.Column(name='Mass', array=np.array(mass3), format='D', unit='Solar Mass')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])

t.writeto('PhaseSpaceBulge_811.fits')

In [48]:
c1 = fits.Column(name='X', array=np.array(pos2[:,0]), format='D', unit = 'kpc')
c2 = fits.Column(name='Y', array=np.array(pos2[:,1]), format='D', unit = 'kpc')
c3 = fits.Column(name='Z', array=np.array(pos2[:,2]), format='D', unit ='kpc')
c4 = fits.Column(name='U', array=np.array(vel2[:,0]), format='D', unit = 'km/s')
c5 = fits.Column(name='V', array=np.array(vel2[:,1]), format='D', unit = 'km/s')
c6 = fits.Column(name='W', array=np.array(vel2[:,2]), format='D', unit='km/s')
c7 = fits.Column(name='Mass', array=np.array(mass2), format='D', unit='Solar Mass')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])

t.writeto('PhaseSpaceDisk_811.fits')

In [49]:
c1 = fits.Column(name='X', array=np.array(pos1[:,0]), format='D', unit = 'kpc')
c2 = fits.Column(name='Y', array=np.array(pos1[:,1]), format='D', unit = 'kpc')
c3 = fits.Column(name='Z', array=np.array(pos1[:,2]), format='D', unit ='kpc')
c4 = fits.Column(name='U', array=np.array(vel1[:,0]), format='D', unit = 'km/s')
c5 = fits.Column(name='V', array=np.array(vel1[:,1]), format='D', unit = 'km/s')
c6 = fits.Column(name='W', array=np.array(vel1[:,2]), format='D', unit='km/s')
c7 = fits.Column(name='Mass', array=np.array(mass1), format='D', unit='Solar Mass')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])

t.writeto('PhaseSpaceDM_811.fits')

In [50]:
c1 = fits.Column(name='X', array=np.array(bulg_pos_019[:,0]), format='D', unit = 'kpc')
c2 = fits.Column(name='Y', array=np.array(bulg_pos_019[:,1]), format='D', unit = 'kpc')
c3 = fits.Column(name='Z', array=np.array(bulg_pos_019[:,2]), format='D', unit ='kpc')
c4 = fits.Column(name='U', array=np.array(bulg_vel_019[:,0]), format='D', unit = 'km/s')
c5 = fits.Column(name='V', array=np.array(bulg_vel_019[:,1]), format='D', unit = 'km/s')
c6 = fits.Column(name='W', array=np.array(bulg_vel_019[:,2]), format='D', unit='km/s')
c7 = fits.Column(name='Mass', array=np.array(bulg_mass_019), format='D', unit='Solar Mass')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])

t.writeto('PhaseSpaceBulge_019.fits')

In [51]:
c1 = fits.Column(name='X', array=np.array(disk_pos_019[:,0]), format='D', unit = 'kpc')
c2 = fits.Column(name='Y', array=np.array(disk_pos_019[:,1]), format='D', unit = 'kpc')
c3 = fits.Column(name='Z', array=np.array(disk_pos_019[:,2]), format='D', unit ='kpc')
c4 = fits.Column(name='U', array=np.array(disk_vel_019[:,0]), format='D', unit = 'km/s')
c5 = fits.Column(name='V', array=np.array(disk_vel_019[:,1]), format='D', unit = 'km/s')
c6 = fits.Column(name='W', array=np.array(disk_vel_019[:,2]), format='D', unit='km/s')
c7 = fits.Column(name='Mass', array=np.array(disk_mass_019), format='D', unit='Solar Mass')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])

t.writeto('PhaseSpaceDisk_019.fits')

In [52]:
c1 = fits.Column(name='X', array=np.array(dm_pos_019[:,0]), format='D', unit = 'kpc')
c2 = fits.Column(name='Y', array=np.array(dm_pos_019[:,1]), format='D', unit = 'kpc')
c3 = fits.Column(name='Z', array=np.array(dm_pos_019[:,2]), format='D', unit ='kpc')
c4 = fits.Column(name='U', array=np.array(dm_vel_019[:,0]), format='D', unit = 'km/s')
c5 = fits.Column(name='V', array=np.array(dm_vel_019[:,1]), format='D', unit = 'km/s')
c6 = fits.Column(name='W', array=np.array(dm_vel_019[:,2]), format='D', unit='km/s')
c7 = fits.Column(name='Mass', array=np.array(dm_mass_019), format='D', unit='Solar Mass')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])

t.writeto('PhaseSpaceDM_019.fits')